In [2]:
import pandas as pd
import os
df = pd.read_csv("/home/zo2151/imagenette/noisy_imagenette.csv")
df_train = df.iloc[:9469,[0,2]]
df_test = df.iloc[9469:,:2]
df_train.to_csv("/home/zo2151/imagenette2-160/train/train.csv")
df_test.to_csv("/home/zo2151/imagenette2-160/val/val.csv")
ptrain = []
ltrain = []
for i in range(9469):
    filename = '/home/zo2151/imagenette2-160/'+df.iloc[i,0]
    filename = filename[:-4]
    filename = filename + "jpeg"
    ptrain.append(filename)
    ltrain.append(df.iloc[i,1])
pval = []
lval = []
for i in range(9469,df.shape[0]):
    filename = '/home/zo2151/imagenette2-160/'+df.iloc[i,0]
    filename = filename[:-4]
    filename = filename + "jpeg"
    pval.append(filename)
    lval.append(df.iloc[i,1])

In [3]:
import torch
import numpy as np
from datasets import load_dataset, load_metric
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )
_val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )
trainset = load_dataset(path = "/home/zo2151/imagenette2-160/train", data_files = ptrain)
valset = load_dataset(path = "/home/zo2151/imagenette2-160/val", data_files = pval)
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


Resolving data files:   0%|          | 0/9469 [00:00<?, ?it/s]

Using custom data configuration train-e4e3d159f28f8ce2
Reusing dataset image_folder (/home/zo2151/.cache/huggingface/datasets/image_folder/train-e4e3d159f28f8ce2/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091)


  0%|          | 0/1 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3925 [00:00<?, ?it/s]

Using custom data configuration val-e3d05b6a6ee52987
Reusing dataset image_folder (/home/zo2151/.cache/huggingface/datasets/image_folder/val-e3d05b6a6ee52987/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
trainset1 = trainset["train"]
test_ds = valset["train"]
splits = trainset1.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [5]:
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)
test_ds.set_transform(val_transforms)

In [6]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  num_labels=10,
                                                )

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
metric_name = "accuracy"
args = TrainingArguments(
    f"test-cifar-10",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

metric = load_metric(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8522
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 402


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.960898,0.976769
2,No log,0.576596,0.975713
3,No log,0.507375,0.976769


***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-134
Configuration saved in test-cifar-10/checkpoint-134/config.json
Model weights saved in test-cifar-10/checkpoint-134/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-134/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-268
Configuration saved in test-cifar-10/checkpoint-268/config.json
Model weights saved in test-cifar-10/checkpoint-268/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-268/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-402
Configuration saved in test-cifar-10/checkpoint-402/config.json
Model weights saved in test-cifar-10/checkpoint-402/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-402/pr

TrainOutput(global_step=402, training_loss=0.9928427928715796, metrics={'train_runtime': 156.3708, 'train_samples_per_second': 163.496, 'train_steps_per_second': 2.571, 'total_flos': 1.9813022585849364e+18, 'train_loss': 0.9928427928715796, 'epoch': 3.0})

In [8]:
outputs = trainer.predict(test_ds)
print(outputs.metrics)

***** Running Prediction *****
  Num examples = 3925
  Batch size = 64


{'test_loss': 0.9502343535423279, 'test_accuracy': 0.9696815286624204, 'test_runtime': 13.9094, 'test_samples_per_second': 282.183, 'test_steps_per_second': 4.457}
